In [1]:
from finalproject.class_dataset import ChestDataset
import pandas as pd
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.applications import DenseNet121
from keras import models
import tensorflow as tf

Using TensorFlow backend.


In [2]:
csvfile = 'finalproject/data/Data_Entry_2017.csv'
train_list = open('finalproject/data/train_val_list.txt','r').read().split('\n')
test_list = open('finalproject/data/test_list.txt','r').read().split('\n')

In [3]:
df = pd.read_csv(csvfile)

In [4]:
df_uni = df[~df['Finding Labels'].str.contains('\|')]

In [5]:
data_dir = 'image/images/'
dataset = ChestDataset(data_dir,df_uni,4)
dataset.reset_folder()
dataset = ChestDataset(data_dir,df_uni,4)
idx = [i for i,el in enumerate(dataset.exists) if el == True]
labels = [dataset.labels[i] for i, el in enumerate(dataset.exists) if el == True]
train_dt,test_dt = dataset.train_test(train_list,test_list)
train_dt.create_tree()
test_dt.create_tree()

/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.reader['exists'] = self.exists
/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:139: UserWarning: file not found
  warnings.warn('file not found')
/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:114: UserWarning: Image not found in folder
  warnings.warn('Image not found in folder')


In [6]:
# # Add GPU options
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
# sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
# NAME = 'Binary_chest_cnn_64x3_{}'.format(int(time.time()))
# tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# #tensorboard --logdir=logs/ --host localhost --port 8088

In [7]:
train_files = train_dt.image_path
test_files = test_dt.image_path
train_folder = train_dt.dir
test_folder = test_dt.dir

In [8]:
label_train = [train_dt.labels[i] for i, el in enumerate(train_dt.exists) if el == True]
label_test = [test_dt.labels[i] for i, el in enumerate(test_dt.exists) if el == True]
print('Train # No Finding:',label_train.count('No Finding')/len(label_train))
print('Test # No Finding:',label_test.count('No Finding')/len(label_test))

Train # No Finding: 0.7090673094752457
Test # No Finding: 0.5034446210916799


In [9]:
print('Statistics about the Dataset:\n')
print('There are %d total chest deseases.' % len(set(dataset.labels)))
print('There are %s total chest images.\n' % np.sum(dataset.exists))
print('There are %d training chest images.' % np.sum(train_dt.exists))
# print('There are %d validation dog images.' % len(valid_files))
print('There are %d test chest images.'% np.sum(test_dt.exists))
for lab in set(labels):
    print('# of %s: %.3f%%'%(lab,100*labels.count(lab)/len(labels)))

Statistics about the Dataset:

There are 15 total chest deseases.
There are 12673 total chest images.

There are 10786 training chest images.
There are 1887 test chest images.
# of Infiltration: 8.672%
# of Mass: 1.941%
# of Effusion: 3.803%
# of Hernia: 0.221%
# of Fibrosis: 1.452%
# of Emphysema: 0.955%
# of Pneumothorax: 1.712%
# of Consolidation: 1.499%
# of Pneumonia: 0.308%
# of Atelectasis: 4.569%
# of Pleural_Thickening: 1.278%
# of Edema: 0.529%
# of No Finding: 67.845%
# of Nodule: 3.133%
# of Cardiomegaly: 2.083%


In [10]:
img_width,img_height = 365,365
densenet = DenseNet121(weights='imagenet', include_top=False,input_shape = (img_width, img_height, 3))

# Freeze some layers
for layer in densenet.layers[:]:
    layer.trainable = False
    
# Create the model
model = models.Sequential()

# Add the vgg convolutional base model

model.add(densenet)

# Add new layers
model.add(Flatten())
# model.add(Dense(72))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.248))
model.add(Dense(15, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 11, 11, 1024)      7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 123904)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                1858575   
Total params: 8,896,079
Trainable params: 1,858,575
Non-trainable params: 7,037,504
_________________________________________________________________


In [ ]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Change the batchsize according to your system RAM
train_batchsize = 10
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(img_height, img_width),
    batch_size=train_batchsize,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    test_folder,
    target_size=(img_height, img_width),
    batch_size=val_batchsize,
    class_mode='categorical',
    shuffle=False)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    verbose=1,
    use_multiprocessing=True)

prediction = model.predict_generator(validation_generator,
                                    validation_generator.samples // validation_generator.batch_size,
                                    verbose=1,
                                    use_multiprocessing=True)



Found 10786 images belonging to 15 classes.
Found 1887 images belonging to 15 classes.
Epoch 1/2
 208/1078 [====>.........................] - ETA: 1:58:34 - loss: 0.6356 - acc: 0.9601